*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
import seaborn as sns

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
#EDA
dataset.plot(kind='scatter', x='age', y='expenses')


In [ ]:
dataset.plot(kind='scatter', x='bmi', y='expenses')

In [ ]:
sns.boxplot(x="smoker", y="expenses", data=dataset)
plt.show()


In [ ]:
sns.boxplot(x="sex", y="expenses", data=dataset)
plt.show()

In [ ]:
sns.boxplot(x="children", y="expenses", data=dataset)
plt.show()

In [ ]:
sns.boxplot(x="region", y="expenses", data=dataset)
plt.show()

In [ ]:
plt.hist(dataset['age'])

In [ ]:
plt.hist(dataset['bmi'])

In [ ]:
plt.hist(dataset['children'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
dataset["sex"] = labelEncoder.fit_transform(dataset["sex"])
dataset["smoker"] = labelEncoder.fit_transform(dataset["smoker"])
dataset["region"] = labelEncoder.fit_transform(dataset["region"])

In [ ]:
X = dataset.iloc[:,:6]
X

In [ ]:
from sklearn.preprocessing import StandardScaler
stdscl = StandardScaler()
X = stdscl.fit_transform(X)

In [ ]:
y = dataset.iloc[:,6]
y

In [ ]:
# Split data into train and test.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state= 100)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train.shape[1]

In [ ]:
model = tf.keras.models.Sequential()


model.add(tf.keras.layers.Dense(units = 15, activation = 'relu', input_shape = (X_train.shape[1],))) # input + hidden layer
model.add(tf.keras.layers.Dense(units = 15, activation = "relu")) # hidden layer
model.add(tf.keras.layers.Dense(units = 15, activation = "relu")) # hidden layer
model.add(tf.keras.layers.Dense(units = 1, activation = "relu")) # ouptput layer

In [ ]:
model.compile(optimizer="adam",
             loss = "mean_absolute_error",
             metrics=['mae', 'mse'])

In [ ]:
mdl_history = model.fit(X_train,
         y_train,
         epochs = 500,
         validation_split=0.2)

In [ ]:
model.summary()

In [ ]:
print(mdl_history.history.keys())

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  #plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(mdl_history)

In [ ]:
# predict the expenses
prediction = model.predict(X_test)

In [ ]:
test_dataset = X_test
test_labels = y_test

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
